# Hyperparameter Tuning ANN

Guidelines to finding the optimal number of Hidden Layers & Hidden Neurons in ANN

- Start simple
- Grid Search / Random Search
- Cross Validation
- Heuristics & Rules of Thumb
  - Number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer
  - Start with 1 - 2 Hidden Layers

In [1]:
import pandas as pd

# Import Dataset

In [2]:
df0 = pd.read_csv('Customer-Churn-Records.csv')
df = df0.copy() 

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   RowNumber           10000 non-null  int64  
 1   CustomerId          10000 non-null  int64  
 2   Surname             10000 non-null  object 
 3   CreditScore         10000 non-null  int64  
 4   Geography           10000 non-null  object 
 5   Gender              10000 non-null  object 
 6   Age                 10000 non-null  int64  
 7   Tenure              10000 non-null  int64  
 8   Balance             10000 non-null  float64
 9   NumOfProducts       10000 non-null  int64  
 10  HasCrCard           10000 non-null  int64  
 11  IsActiveMember      10000 non-null  int64  
 12  EstimatedSalary     10000 non-null  float64
 13  Exited              10000 non-null  int64  
 14  Complain            10000 non-null  int64  
 15  Satisfaction Score  10000 non-null  int64  
 16  Card 

In [4]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0,0,1,DIAMOND,300
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0,0,5,PLATINUM,771
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1,1,3,SILVER,564
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1,1,2,GOLD,339


---

## Data Cleaning

In [5]:
df.columns = df.columns.str.lower()

In [6]:
df.columns.tolist()

['rownumber',
 'customerid',
 'surname',
 'creditscore',
 'geography',
 'gender',
 'age',
 'tenure',
 'balance',
 'numofproducts',
 'hascrcard',
 'isactivemember',
 'estimatedsalary',
 'exited',
 'complain',
 'satisfaction score',
 'card type',
 'point earned']

In [7]:
df.columns = df.columns.str.replace(' ', '_')

In [8]:
df.columns.tolist()

['rownumber',
 'customerid',
 'surname',
 'creditscore',
 'geography',
 'gender',
 'age',
 'tenure',
 'balance',
 'numofproducts',
 'hascrcard',
 'isactivemember',
 'estimatedsalary',
 'exited',
 'complain',
 'satisfaction_score',
 'card_type',
 'point_earned']

In [9]:
cols_to_rename = {
    'geography':'country',
    'numofproducts':'num_of_products',
    'hascrcard':'has_creditcard',
    'isactivemember':'is_active',
    'estimatedsalary':'salary',
    'complain':'complained'
}

In [10]:
df.rename(columns=cols_to_rename, inplace=True)

In [11]:
df.columns.tolist()

['rownumber',
 'customerid',
 'surname',
 'creditscore',
 'country',
 'gender',
 'age',
 'tenure',
 'balance',
 'num_of_products',
 'has_creditcard',
 'is_active',
 'salary',
 'exited',
 'complained',
 'satisfaction_score',
 'card_type',
 'point_earned']

---

## Feature Engineering

In [12]:
df = df.drop(['rownumber','customerid','surname','card_type'], axis=1)

In [13]:
df

,creditscore,country,gender,age,tenure,balance,num_of_products,has_creditcard,is_active,salary,exited,complained,satisfaction_score,point_earned
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,464
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,456
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,377
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,350
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0,0,1,300
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0,0,5,771
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1,1,3,564
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1,1,2,339


---

## Data Encoding

In [14]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [15]:
lbl_encoder = LabelEncoder()

In [16]:
df['gender'] = lbl_encoder.fit_transform(df['gender'])

In [17]:
df

,creditscore,country,gender,age,tenure,balance,num_of_products,has_creditcard,is_active,salary,exited,complained,satisfaction_score,point_earned
0,619,France,0,42,2,0.00,1,1,1,101348.88,1,1,2,464
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0,1,3,456
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1,1,3,377
3,699,France,0,39,1,0.00,2,0,0,93826.63,0,0,5,350
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0,0,5,425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0,0,1,300
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0,0,5,771
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1,1,3,564
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1,1,2,339


In [18]:
oh_encoder = OneHotEncoder()

In [19]:
country_encoded = oh_encoder.fit_transform(df[['country']]).toarray()

In [20]:
col_names = oh_encoder.get_feature_names_out(['country'])
col_names

array(['country_France', 'country_Germany', 'country_Spain'], dtype=object)

In [21]:
country_encoded_df = pd.DataFrame(country_encoded, columns=col_names)

In [22]:
df = pd.concat([df.drop('country',axis=1), country_encoded_df], axis=1)

In [23]:
df.head()

,creditscore,gender,age,tenure,balance,num_of_products,has_creditcard,is_active,salary,exited,complained,satisfaction_score,point_earned,country_France,country_Germany,country_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1,2,464,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,1,3,456,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1,3,377,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0,5,350,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,5,425,0.0,0.0,1.0


In [24]:
X = df.drop('exited', axis=1)

In [25]:
y = df['exited']

In [26]:
X.shape

(10000, 15)

In [27]:
y.shape

(10000,)

## Train Test Split

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [30]:
X_train.shape, y_train.shape

((8000, 15), (8000,))

In [31]:
X_test.shape, y_test.shape

((2000, 15), (2000,))

---

# Scale Data

In [32]:
from sklearn.preprocessing import StandardScaler

In [33]:
scaler = StandardScaler()

In [34]:
X_train = scaler.fit_transform(X_train)

In [35]:
X_test = scaler.transform(X_test)

In [36]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [37]:
X_test

array([[-0.57749609,  0.91324755, -0.6557859 , ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.29729735,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.52560743, -1.09499335,  0.48508334, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.81311987, -1.09499335,  0.77030065, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.41876609,  0.91324755, -0.94100321, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.24540869,  0.91324755,  0.00972116, ..., -0.99850112,
         1.72572313, -0.57638802]])

---

# Export Components

In [38]:
import pickle

In [39]:
with open('lbl_encoder.pkl','wb') as file:
    pickle.dump(lbl_encoder, file)

In [40]:
with open('oh_encoder','wb') as file:
    pickle.dump(oh_encoder, file)

In [41]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)

---

# Train Model

In [43]:
import tensorflow as tf
import tensorflow.keras as keras
import keras

In [44]:
print(tf.__version__)

2.15.0


In [45]:
print(keras.__version__)

3.12.0


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input 

In [47]:
def create_model(neurons=32, layers=1, optimizer='adam', loss='binary_crossentropy'):
    # Determine the number of hidden layers to add after the first one
    num_additional_hidden_layers = layers - 1 

    # 1. Define the first layer (Input and first Dense layer combined)
    model_layers = [
        Input(shape=(X_train.shape[1],)),
        Dense(neurons, activation='relu')
    ]

    # 2. Use a list comprehension to add the additional hidden layers
    # This correctly generates the layers as a list of Dense objects
    additional_hidden_layers = [
        Dense(neurons, activation='relu') 
        for _ in range(num_additional_hidden_layers)
    ]

    # 3. Concatenate all layers: Initial layer(s) + Additional hidden layers
    # This also handles the case where layers=1 (no additional hidden layers are added)
    model_layers.extend(additional_hidden_layers)

    # 4. Add the output layer (Always the last layer)
    model_layers.append(Dense(1, activation='sigmoid'))

    # 5. Create the Sequential model from the consolidated list
    model = Sequential(model_layers)


    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    
    return model

### Create KerasClassifier

In [48]:
from scikeras.wrappers import KerasClassifier

In [50]:
model = KerasClassifier(model=create_model, verbose=0)

In [51]:
model

,model,<function cre...x7e4d765e8b80>
,build_fn,None
,warm_start,False
,random_state,None
,optimizer,'rmsprop'
,loss,None
,metrics,None
,batch_size,None
,validation_batch_size,None
,verbose,0
,callbacks,None


# Parameter Tuning

In [53]:
param_grid = {
    # Custom model function parameters (prefixed with 'model__'):
    'model__neurons': [16, 32, 64],
    'model__layers': [1, 2, 3],
    'model__optimizer': ['adam'],
    'model__loss': ['binary_crossentropy'],

    # Wrapper/Fitting parameters (no prefix):
    # 'batch_size': [10, 20],
    # 'epochs': [50, 100]
}

### GridSearchCV

In [54]:
from sklearn.model_selection import GridSearchCV

In [55]:
gridsearchCV = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3, error_score='raise')

In [56]:
grid_result = gridsearchCV.fit(X_train, y_train)

2025-11-28 18:44:15.191611: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:c2:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-11-28 18:44:15.334184: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [57]:
print('Best: %f using %s' % (grid_result.best_score_, grid_result.best_params_))

Best: 0.998500 using {'model__layers': 2, 'model__loss': 'binary_crossentropy', 'model__neurons': 64, 'model__optimizer': 'adam'}


---